In [1]:
%load_ext autoreload
%load_ext autotime

time: 422 µs (started: 2023-06-14 16:28:40 -07:00)


In [2]:
import sys
sys.path.append("../../")

time: 636 µs (started: 2023-06-14 16:28:40 -07:00)


In [102]:
%%writefile test.py

# import resource

# rsrc = resource.RLIMIT_DATA
# soft, hard = resource.getrlimit(rsrc)
# print('Soft limit starts as  :', soft)

# resource.setrlimit(rsrc, (4 * (1024 ** 3), hard)) #limit to one kilobyte

# soft, hard = resource.getrlimit(rsrc)
# print('Soft limit changed to :', soft)

import dask.dataframe as dd
import itertools as it
import pandas as pd
from time import time

# rsrc = resource.RLIMIT_DATA
# soft, hard = resource.getrlimit(rsrc)
# resource.setrlimit(rsrc, (1024 * 1024 * 765, hard))
# soft, hard = resource.getrlimit(rsrc)

indices = ['file_name', 'proc_name', 'io_cat', 'acc_pat']
combinations = list(it.combinations(indices, r=2))[:5]

def simple_pd(log_dir):
    print(f",PD")
    df = pd.read_parquet(log_dir, engine='pyarrow')
    query_index = 0
    for ix in ['file_name', 'proc_name']:
        for col in ['size', 'duration', 'index', 'bandwidth', 'io_cat']: 
            query_index += 1 
            t1 = time()
            df.groupby([ix]).agg({col: 'sum'})
            time_elapsed = time() - t1
            print(f"simple,Q{query_index},{time_elapsed}")


def simple_dask(log_dir):
    print(f",DASK")
    query_index = 0
    ddf = dd.read_parquet(f"{log_dir}/*.parquet", engine='pyarrow')
    for ix in ['file_name', 'proc_name']:
        for col in ['size', 'duration', 'index', 'bandwidth', 'io_cat']:
            query_index += 1 
            t1 = time()
            ddf.groupby([ix]).agg({col: 'sum'}).compute()
            time_elapsed = time() - t1
            print(f"{time_elapsed}")


def simple_wisio_pd(log_dir):
    print(f",WISIO PD")
    query_index = 0
    df = pd.read_parquet(log_dir, engine='pyarrow')
    cols = ['size', 'duration', 'index', 'bandwidth', 'io_cat']
    agg_dict = {col: sum for col in cols}
    for ix in ['file_name', 'proc_name']:
        for col in cols:
            query_index += 1 
            t1 = time()
            if query_index == 1:
                x = df.groupby(['file_name', 'proc_name']).agg(agg_dict)
            x.groupby([ix]).agg({col: 'sum'})
            time_elapsed = time() - t1
            print(f"{time_elapsed}")


def simple_wisio_dask(log_dir):
    print(f",WISIO DASK")
    query_index = 0
    ddf = dd.read_parquet(f"{log_dir}/*.parquet", engine='pyarrow')
    cols = ['size', 'duration', 'index', 'bandwidth', 'io_cat']
    agg_dict = {col: sum for col in cols}
    for ix in ['file_name', 'proc_name']:
        for col in cols:
            query_index += 1 
            t1 = time()
            if query_index == 1:
                x = ddf.groupby(['file_name', 'proc_name']).agg(agg_dict).compute()
            x.groupby([ix]).agg({col: 'sum'})
            time_elapsed = time() - t1
            print(f"{time_elapsed}")

def medium_pd(log_dir):
    print(f",PD")
    df = pd.read_parquet(log_dir, engine='pyarrow')
    query_index = 0
    for ix, iy in combinations:
        for col in ['size', 'duration']:
            query_index += 1 
            t1 = time()
            df.groupby([ix, iy]).agg({col: 'sum'}).groupby([ix]).sum()
            time_elapsed = time() - t1
            print(f"medium,Q{query_index},{time_elapsed}")
            

def medium_dask(log_dir):
    print(f",DASK")
    ddf = dd.read_parquet(f"{log_dir}/*.parquet", engine='pyarrow')
    query_index = 0
    for ix, iy in combinations:
        for col in ['size', 'duration']:
            query_index += 1 
            t1 = time()
            ddf.groupby([ix, iy]).agg({col: 'sum'}).groupby([ix]).sum().compute()
            time_elapsed = time() - t1
            print(f"{time_elapsed}")

def medium_wisio_pd(log_dir):
    print(f",WISIO PD")
    df = pd.read_parquet(log_dir, engine='pyarrow')
    query_index = 0
    for ix, iy in combinations:
        for col in ['size', 'duration']:
            query_index += 1 
            t1 = time()
            if query_index == 1:
                x = df.groupby(indices).agg({'size': sum, 'duration': sum})
            x.groupby([ix, iy]).agg({col: 'sum'}).groupby([ix]).sum()   
            time_elapsed = time() - t1
            print(f"{time_elapsed}")

def medium_wisio_dask(log_dir):
    print(f",WISIO DASK")
    ddf = dd.read_parquet(f"{log_dir}/*.parquet", engine='pyarrow')
    query_index = 0
    for ix, iy in combinations:
        for col in ['size', 'duration']:
            query_index += 1 
            t1 = time()
            if query_index == 1:
                x = ddf.groupby(indices).agg({'size': sum, 'duration': sum}).compute()
            x.groupby([ix, iy]).agg({col: 'sum'}).groupby([ix]).sum()
            time_elapsed = time() - t1
            print(f"{time_elapsed}")

def hard_pd(log_dir):
    print(f",PD")
    df = pd.read_parquet(log_dir, engine='pyarrow')
    query_index = 0
    
    # groupby(id1).agg(id2:list,col:sum) groupby(id2).agg(id1:list, col:sum) explode merge agg

Overwriting test.py
time: 4.99 ms (started: 2023-06-14 18:06:00 -07:00)


In [103]:
%%writefile -a test.py

import sys

log_dir = sys.argv[1]
log_index = sys.argv[2]

for test_type in [
    simple_pd, 
    simple_dask, 
    simple_wisio_pd, 
    simple_wisio_dask, 
    medium_pd, 
    medium_dask, 
    medium_wisio_pd, 
    medium_wisio_dask
]:
    print(f",{log_index}")
    # try:
    test_type(log_dir=log_dir)
    # except:
    #     print('\n'*10)
    #     continue



Appending to test.py
time: 2.8 ms (started: 2023-06-14 18:06:05 -07:00)


500k Records

In [64]:
!which python

~/projects/vani-analysis-tool/venv-quartz2/bin/python


time: 283 ms (started: 2023-06-14 17:42:35 -07:00)


In [65]:
%%timeit -r 10

!python test.py '../tmp/eval-samples/data-reduction-500k-dask' '500k'

['/g/g92/izzet/projects/vani-analysis-tool/notebooks/evals', '/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python39.zip', '/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python3.9', '/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python3.9/lib-dynload', '/g/g92/izzet/projects/vani-analysis-tool/venv-quartz2/lib/python3.9/site-packages']
['/g/g92/izzet/projects/vani-analysis-tool/notebooks/evals', '/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python39.zip', '/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python3.9', '/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52

In [16]:
%%timeit -r 10

!python test.py '../tmp/eval-samples/data-reduction-1m-dask' 'simple_dask'

1.97 s ± 67 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 21.6 s (started: 2023-06-14 16:45:34 -07:00)


In [17]:
%%timeit -r 10

!python test.py '../tmp/eval-samples/data-reduction-1m-dask' 'simple_wisio_pd'

4.47 s ± 353 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 50 s (started: 2023-06-14 16:45:56 -07:00)


In [18]:
%%timeit -r 10

!python test.py '../tmp/eval-samples/data-reduction-1m-dask' 'simple_wisio_dask'

1.98 s ± 147 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 21.8 s (started: 2023-06-14 16:46:46 -07:00)


5m Records

In [19]:
%%timeit -r 10

!python test.py '../tmp/eval-samples/data-reduction-5m-dask' 'simple_pd'

21.5 s ± 782 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 4min 9s (started: 2023-06-14 16:47:56 -07:00)


In [29]:
%%timeit -r 10

!python test.py '../tmp/eval-samples/data-reduction-5m-dask' 'simple_dask'

58.1 s ± 1.64 s per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 10min 36s (started: 2023-06-14 17:04:45 -07:00)


In [21]:
%%timeit -r 10

!python test.py '../tmp/eval-samples/data-reduction-5m-dask' 'simple_wisio_pd'

15.6 s ± 1.03 s per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 2min 51s (started: 2023-06-14 16:52:27 -07:00)


In [26]:
%%timeit -r 10

!python test.py '../tmp/eval-samples/data-reduction-5m-dask' 'simple_wisio_dask'

8.16 s ± 167 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 1min 30s (started: 2023-06-14 17:01:56 -07:00)


## Dask

In [3]:
%autoreload 2

from wisio.recorder import RecorderAnalyzer
from wisio.rules import Rule
from wisio._recorder.rules import RecorderRuleEngine

recorder_analyzer = RecorderAnalyzer(
    cluster_manager_args=dict(
        force_local=True,
        cluster_settings=dict(
            cores=4,
            dashboard_port=3660,
            local_directory="/var/tmp/dask-recorder",
            log_file="%J.log",
            worker_queue="pdebug"
        )
    ),
    working_dir='.recorder',
    debug=True
) 

recorder_analyzer

distributed.diskutils - INFO - Found stale lock file and directory '/var/tmp/dask-recorder/cluster-0-local/dask-worker-space/worker-qv85v2gp', purging
distributed.diskutils - INFO - Found stale lock file and directory '/var/tmp/dask-recorder/cluster-0-local/dask-worker-space/worker-y2mlan68', purging
distributed.diskutils - INFO - Found stale lock file and directory '/var/tmp/dask-recorder/cluster-0-local/dask-worker-space/worker-aujtvm72', purging
distributed.diskutils - INFO - Found stale lock file and directory '/var/tmp/dask-recorder/cluster-0-local/dask-worker-space/worker-34ab2lp1', purging


time: 4.28 s (started: 2023-06-12 12:43:00 -07:00)


In [53]:
def analyze_dask_small_writes(log_dir: str, view_type: str, checkpoint=True):
    main_view, views, bottlenecks = recorder_analyzer.analyze_parquet(
        log_dir=log_dir,
        checkpoint=checkpoint,
        desired_view_names=[view_type],
    )

    rule_engine = RecorderRuleEngine(rules={view_type: [Rule.BOTT_SMALL_WRITES]})

    characteristics = rule_engine.process_characteristics(view=main_view)

    processed_bottlenecks = rule_engine.process_bottlenecks(bottlenecks=bottlenecks, characteristics=characteristics, threshold=0.5)

    return main_view, views, characteristics, processed_bottlenecks

def analyze_dask_metadata_access(log_dir: str, view_type: str, checkpoint=True):
    main_view, views, bottlenecks = recorder_analyzer.analyze_parquet(
        log_dir=log_dir,
        checkpoint=checkpoint,
        desired_view_names=[view_type],
    )

    rule_engine = RecorderRuleEngine(rules={view_type: [Rule.BOTT_METADATA_ACCESS]})

    characteristics = rule_engine.process_characteristics(view=main_view)

    processed_bottlenecks = rule_engine.process_bottlenecks(bottlenecks=bottlenecks, characteristics=characteristics, threshold=0.01)

    return main_view, views, characteristics, processed_bottlenecks

time: 1.8 ms (started: 2023-06-12 13:38:56 -07:00)


### File-based - Small Writes - 1m Records

In [20]:
!du -ah ../tmp/eval-samples/data-reduction-1m-dask

44M	../tmp/eval-samples/data-reduction-1m-dask/part.1.parquet
40M	../tmp/eval-samples/data-reduction-1m-dask/part.5.parquet
44M	../tmp/eval-samples/data-reduction-1m-dask/part.0.parquet
43M	../tmp/eval-samples/data-reduction-1m-dask/part.4.parquet
0	../tmp/eval-samples/data-reduction-1m-dask/global.json
44M	../tmp/eval-samples/data-reduction-1m-dask/part.3.parquet
43M	../tmp/eval-samples/data-reduction-1m-dask/part.2.parquet
16K	../tmp/eval-samples/data-reduction-1m-dask/_metadata
4.0K	../tmp/eval-samples/data-reduction-1m-dask/_common_metadata
740K	../tmp/eval-samples/data-reduction-1m-dask/checkpoints/_main_view/part.0.parquet
12K	../tmp/eval-samples/data-reduction-1m-dask/checkpoints/_main_view/_metadata
8.0K	../tmp/eval-samples/data-reduction-1m-dask/checkpoints/_main_view/_common_metadata
764K	../tmp/eval-samples/data-reduction-1m-dask/checkpoints/_main_view
700K	../tmp/eval-samples/data-reduction-1m-dask/checkpoints/file_name/part.0.parquet
12K	../tmp/eval-samples/data-reduction-

#### Without Checkpointing

In [38]:
%%timeit -r 10

main_view, views, characteristics, bottlenecks = analyze_dask_small_writes(log_dir='../tmp/eval-samples/data-reduction-1m-dask', view_type='file_name', checkpoint=False)

/g/g92/izzet/projects/vani-analysis-tool/notebooks/evals/../../wisio/_recorder/rules.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xfer_bins[per_col] = xfer_bins[count_col] / xfer_bins[count_col].sum()


2.24 s ± 430 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 25.6 s (started: 2023-06-12 13:31:07 -07:00)


#### With Checkpointing

Create checkpoints

In [39]:
main_view, views, characteristics, bottlenecks = analyze_dask_small_writes(log_dir='../tmp/eval-samples/data-reduction-1m-dask', view_type='file_name', checkpoint=True)

time: 741 ms (started: 2023-06-12 13:33:15 -07:00)


Do the actual analysis

In [5]:
%%timeit -r 10

main_view, views, characteristics, bottlenecks = analyze_dask_small_writes(log_dir='../tmp/eval-samples/data-reduction-1m-dask', view_type='file_name')

/g/g92/izzet/projects/vani-analysis-tool/notebooks/evals/../../wisio/_recorder/rules.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xfer_bins[per_col] = xfer_bins[count_col] / xfer_bins[count_col].sum()


909 ms ± 235 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 14.3 s (started: 2023-06-12 12:43:04 -07:00)


List bottlenecks

In [35]:
main_view, views, characteristics, bottlenecks = analyze_dask_small_writes(log_dir='../tmp/eval-samples/data-reduction-1m-dask', view_type='file_name')

bottlenecks

{('file_name',): {<Rule.BOTT_SMALL_WRITES: 212>: {'/p/gpfs1/iopp/temp/montage/2854964/5/run/ngc3372.fits': RuleResult(data_dict={'llc': {'duration_sum': 0.56511009982205, 'index_count': 1888, 'size_min': 172352, 'size_max': 172352, 'size_sum': 324021760, 'read_time': 0.0, 'write_time': 0.5644265998198534, 'metadata_time': 0.0006835000021965243, 'read_size': 0, 'write_size': 324021760, 'metadata_size': 0, 'read_count': 0, 'write_count': 1880, 'metadata_count': 8, 'read_min': 0, 'read_max': 0, 'write_min': 344704, 'write_max': 344704, 'metadata_min': 0, 'metadata_max': 0, 'data_count': 1880, 'data_size': 324021760, 'data_time': 0.5644265998198534, 'sequential_time': 0.5644265998198534, 'random_time': 0.0, 'sequential_size': 324021760, 'random_size': 0, 'sequential_count': 1880, 'random_count': 0, 'close_time': 0.0003333999993628822, 'open_time': 0.00035010000283364207, 'seek_time': 0.0, 'stat_time': 0.0, 'close_count': 4, 'open_count': 4, 'seek_count': 0, 'stat_count': 0, 'duration_pero'

time: 959 ms (started: 2023-06-12 13:30:04 -07:00)


### File-based - Small Writes - 5m Records

In [19]:
!du -ah ../tmp/eval-samples/data-reduction-5m-dask

48M	../tmp/eval-samples/data-reduction-5m-dask/part.0.parquet
48M	../tmp/eval-samples/data-reduction-5m-dask/part.16.parquet
46M	../tmp/eval-samples/data-reduction-5m-dask/part.4.parquet
48M	../tmp/eval-samples/data-reduction-5m-dask/part.8.parquet
49M	../tmp/eval-samples/data-reduction-5m-dask/part.1.parquet
48M	../tmp/eval-samples/data-reduction-5m-dask/part.3.parquet
47M	../tmp/eval-samples/data-reduction-5m-dask/part.5.parquet
48M	../tmp/eval-samples/data-reduction-5m-dask/part.6.parquet
48M	../tmp/eval-samples/data-reduction-5m-dask/part.12.parquet
48M	../tmp/eval-samples/data-reduction-5m-dask/part.9.parquet
48M	../tmp/eval-samples/data-reduction-5m-dask/part.7.parquet
45M	../tmp/eval-samples/data-reduction-5m-dask/part.15.parquet
49M	../tmp/eval-samples/data-reduction-5m-dask/part.11.parquet
48M	../tmp/eval-samples/data-reduction-5m-dask/part.19.parquet
48M	../tmp/eval-samples/data-reduction-5m-dask/part.14.parquet
49M	../tmp/eval-samples/data-reduction-5m-dask/part.17.parquet
4

#### Without Checkpointing

In [40]:
%%timeit -r 10

main_view, views, characteristics, bottlenecks = analyze_dask_small_writes(log_dir='../tmp/eval-samples/data-reduction-5m-dask', view_type='file_name', checkpoint=False)

5.22 s ± 352 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 58.9 s (started: 2023-06-12 13:34:25 -07:00)


#### With Checkpointing

Create checkpoints

In [42]:
main_view, views, characteristics, bottlenecks = analyze_dask_small_writes(log_dir='../tmp/eval-samples/data-reduction-5m-dask', view_type='file_name', checkpoint=True)

time: 778 ms (started: 2023-06-12 13:35:36 -07:00)


Do the actual analysis

In [43]:
%%timeit -r 10

main_view, views, characteristics, bottlenecks = analyze_dask_small_writes(log_dir='../tmp/eval-samples/data-reduction-5m-dask', view_type='file_name', checkpoint=True)

/g/g92/izzet/projects/vani-analysis-tool/notebooks/evals/../../wisio/_recorder/rules.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xfer_bins[per_col] = xfer_bins[count_col] / xfer_bins[count_col].sum()


783 ms ± 57.7 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 8.96 s (started: 2023-06-12 13:35:39 -07:00)


List bottlenecks

In [44]:
main_view, views, characteristics, bottlenecks = analyze_dask_small_writes(log_dir='../tmp/eval-samples/data-reduction-5m-dask', view_type='file_name', checkpoint=True)

bottlenecks

{('file_name',): {<Rule.BOTT_SMALL_WRITES: 212>: {'/p/gpfs1/iopp/temp/montage/2854964/5/run/ngc3372.fits': RuleResult(data_dict={'llc': {'duration_sum': 2.82555049911025, 'index_count': 9440, 'size_min': 172352, 'size_max': 172352, 'size_sum': 1620108800, 'read_time': 0.0, 'write_time': 2.822132999099267, 'metadata_time': 0.0034175000109826215, 'read_size': 0, 'write_size': 1620108800, 'metadata_size': 0, 'read_count': 0, 'write_count': 9400, 'metadata_count': 40, 'read_min': 0, 'read_max': 0, 'write_min': 344704, 'write_max': 344704, 'metadata_min': 0, 'metadata_max': 0, 'data_count': 9400, 'data_size': 1620108800, 'data_time': 2.822132999099267, 'sequential_time': 2.822132999099267, 'random_time': 0.0, 'sequential_size': 1620108800, 'random_size': 0, 'sequential_count': 9400, 'random_count': 0, 'close_time': 0.0016669999968144111, 'open_time': 0.0017505000141682103, 'seek_time': 0.0, 'stat_time': 0.0, 'close_count': 20, 'open_count': 20, 'seek_count': 0, 'stat_count': 0, 'duration_pe

time: 877 ms (started: 2023-06-12 13:35:48 -07:00)


### Time-based - Metadata Access - 1m Records

#### Without Checkpointing

In [57]:
%%timeit -r 10

main_view, views, characteristics, bottlenecks = analyze_dask_metadata_access(log_dir='../tmp/eval-samples/data-reduction-1m-dask', view_type='trange', checkpoint=False)

2.3 s ± 373 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 25.1 s (started: 2023-06-12 13:40:38 -07:00)


#### With Checkpointing

Create checkpoints

In [58]:
main_view, views, characteristics, bottlenecks = analyze_dask_metadata_access(log_dir='../tmp/eval-samples/data-reduction-1m-dask', view_type='trange', checkpoint=True)

time: 816 ms (started: 2023-06-12 13:41:58 -07:00)


Do the actual analysis

In [59]:
%%timeit -r 10

main_view, views, characteristics, bottlenecks = analyze_dask_metadata_access(log_dir='../tmp/eval-samples/data-reduction-1m-dask', view_type='trange', checkpoint=True)

805 ms ± 46.6 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 8.9 s (started: 2023-06-12 13:42:17 -07:00)


List bottlenecks

In [77]:
main_view, views, characteristics, bottlenecks = analyze_dask_metadata_access(log_dir='../tmp/eval-samples/data-reduction-1m-dask', view_type='trange', checkpoint=True)

bottlenecks

{('trange',): {<Rule.BOTT_METADATA_ACCESS: 207>: {4: RuleResult(data_dict={'llc': {'duration_sum': 0.07102449819046797, 'index_count': 46278, 'size_min': -2, 'size_max': 32768, 'size_sum': 209251785, 'read_time': 0.051251098273610296, 'write_time': 0.019084899927520382, 'metadata_time': 0.04143710181443794, 'read_size': 86837302, 'write_size': 122414699, 'metadata_size': -216, 'read_count': 4308, 'write_count': 40828, 'metadata_count': 1142, 'read_min': 74972, 'read_max': 1044224, 'write_min': 89924, 'write_max': 90034, 'metadata_min': -112, 'metadata_max': 0, 'data_count': 45136, 'data_size': 209252001, 'data_time': 0.07033599820113068, 'sequential_time': 0.07033599820113068, 'random_time': 0.0, 'sequential_size': 209252001, 'random_size': 0, 'sequential_count': 45136, 'random_count': 0, 'close_time': 0.0007828999750927323, 'open_time': 0.04054910180366278, 'seek_time': 0.00018719999889071914, 'stat_time': 0.00028920000022480963, 'close_count': 223, 'open_count': 250, 'seek_count': 16

time: 898 ms (started: 2023-06-12 13:54:27 -07:00)


### Time-based - Metadata Access - 5m Records

#### Without Checkpointing

In [61]:
%%timeit -r 10

main_view, views, characteristics, bottlenecks = analyze_dask_metadata_access(log_dir='../tmp/eval-samples/data-reduction-5m-dask', view_type='trange', checkpoint=False)

5.04 s ± 278 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 56.2 s (started: 2023-06-12 13:43:39 -07:00)


#### With Checkpointing

Create checkpoints

In [62]:
main_view, views, characteristics, bottlenecks = analyze_dask_metadata_access(log_dir='../tmp/eval-samples/data-reduction-5m-dask', view_type='trange', checkpoint=True)

time: 1.74 s (started: 2023-06-12 13:44:51 -07:00)


Do the actual analysis

In [63]:
%%timeit -r 10

main_view, views, characteristics, bottlenecks = analyze_dask_metadata_access(log_dir='../tmp/eval-samples/data-reduction-5m-dask', view_type='trange', checkpoint=True)

/g/g92/izzet/projects/vani-analysis-tool/notebooks/evals/../../wisio/_recorder/rules.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xfer_bins[per_col] = xfer_bins[count_col] / xfer_bins[count_col].sum()
/g/g92/izzet/projects/vani-analysis-tool/notebooks/evals/../../wisio/_recorder/rules.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xfer_bins[per_col] = xfer_bins[count_col] / xfer_bins[count_col].sum()


871 ms ± 65.2 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 9.61 s (started: 2023-06-12 13:45:03 -07:00)


List bottlenecks

In [64]:
main_view, views, characteristics, bottlenecks = analyze_dask_metadata_access(log_dir='../tmp/eval-samples/data-reduction-5m-dask', view_type='trange', checkpoint=True)

bottlenecks

{('trange',): {<Rule.BOTT_METADATA_ACCESS: 207>: {4: RuleResult(data_dict={'llc': {'duration_sum': 0.35512249095233983, 'index_count': 231390, 'size_min': -2, 'size_max': 32768, 'size_sum': 1046258925, 'read_time': 0.2562554913680515, 'write_time': 0.09542449963760191, 'metadata_time': 0.2071855090721897, 'read_size': 434186510, 'write_size': 612073495, 'metadata_size': -1080, 'read_count': 21540, 'write_count': 204140, 'metadata_count': 5710, 'read_min': 74972, 'read_max': 1044224, 'write_min': 89924, 'write_max': 90034, 'metadata_min': -112, 'metadata_max': 0, 'data_count': 225680, 'data_size': 1046260005, 'data_time': 0.3516799910056534, 'sequential_time': 0.3516799910056534, 'random_time': 0.0, 'sequential_size': 1046260005, 'random_size': 0, 'sequential_count': 225680, 'random_count': 0, 'close_time': 0.003914499875463662, 'open_time': 0.2027455090183139, 'seek_time': 0.0009359999944535957, 'stat_time': 0.0014460000011240481, 'close_count': 1115, 'open_count': 1250, 'seek_count': 

time: 1.02 s (started: 2023-06-12 13:45:24 -07:00)


## Pandas

In [78]:
import numpy as np
import pandas as pd
from wisio._recorder.analysis import _set_derived_columns
from wisio._recorder.constants import VIEW_TYPES, LOGICAL_VIEW_TYPES

time: 974 µs (started: 2023-06-12 13:59:05 -07:00)


In [79]:
def _get_agg_dict(view_columns: list, is_proc=False):
    if is_proc:
        agg_dict = {col: max if any(x in col for x in 'duration time'.split()) else sum for col in view_columns}
    else:
        agg_dict = {col: sum for col in view_columns}
    agg_dict['size_min'] = min
    agg_dict['size_max'] = max
    for view_type in [*VIEW_TYPES, *LOGICAL_VIEW_TYPES]:
        if view_type in agg_dict:
            agg_dict.pop(view_type)
    return agg_dict

time: 1.6 ms (started: 2023-06-12 13:59:14 -07:00)


In [93]:
METADATA_ACCESS_RATIO_THRESHOLD = 0.18
METADATA_ACCESS_DURATION_THRESHOLD = 0.01
SMALL_WRITE_DURATION_THRESHOLD = 0.5
SMALL_WRITE_RATIO_THRESHOLD = 0.75
SMALL_WRITE_SIZE_THRESHOLD = 256 * 1024

time: 1.06 ms (started: 2023-06-12 14:07:30 -07:00)


In [94]:
def analyze_pandas_small_writes(log_file: str, view_type: str):
    sample_1m_df = pd.read_parquet(log_file)

    tmid_max = sample_1m_df['tmid'].max()
    tranges = np.arange(0, tmid_max, 1e7)

    sample_1m_df = sample_1m_df[(sample_1m_df['cat'] == 0) & (sample_1m_df['io_cat'].isin([1, 2, 3]))]

    max_io_time = sample_1m_df.groupby(['proc_name']).sum()['duration'].max()

    sample_1m_bot_df = sample_1m_df \
        .assign(trange=lambda x: np.digitize(x['tmid'], bins=tranges, right=True)) \
        .groupby([view_type, 'io_cat', 'acc_pat', 'func_id']) \
        .agg({
            'duration': ['sum'],
            'size': ['min', 'max', 'sum'],
            'index': ['count'],
            'proc_name': [list],
            'trange': [list],
        })

    sample_1m_bot_df.columns = ['_'.join(col) for col in sample_1m_bot_df.columns.values]
    sample_1m_bot_df.reset_index(inplace=True)

    _set_derived_columns(ddf=sample_1m_bot_df)

    sample_1m_bot_df['write_time_ratio'] = sample_1m_bot_df['write_time'] / sample_1m_bot_df['duration_sum']
    sample_1m_bot_df['write_avg_size'] = sample_1m_bot_df['write_size'] / sample_1m_bot_df['index_count']

    sample_1m_bot_df.drop(columns=['io_cat', 'acc_pat', 'func_id'], inplace=True)

    agg_dict = {col: sum for col in sample_1m_bot_df.columns}
    agg_dict['size_min'] = min
    agg_dict['size_max'] = max
    agg_dict.pop(view_type)

    sample_1m_bot_df = sample_1m_bot_df \
        .groupby([view_type]) \
        .agg(agg_dict)

    sample_1m_bot_df['duration_ratio'] = sample_1m_bot_df['duration_sum'] / max_io_time

    return sample_1m_bot_df.query(f"duration_ratio > {SMALL_WRITE_DURATION_THRESHOLD} and write_time_ratio > {SMALL_WRITE_RATIO_THRESHOLD} and write_avg_size < {SMALL_WRITE_SIZE_THRESHOLD}"), max_io_time

def analyze_pandas_metadata_access(log_file: str, view_type: str):
    sample_1m_df = pd.read_parquet(log_file)

    tmid_max = sample_1m_df['tmid'].max()
    tranges = np.arange(0, tmid_max, 1e7)

    sample_1m_df = sample_1m_df[(sample_1m_df['cat'] == 0) & (sample_1m_df['io_cat'].isin([1, 2, 3]))]

    max_io_time = sample_1m_df.groupby(['proc_name']).sum()['duration'].max()

    sample_1m_bot_df = sample_1m_df \
        .assign(trange=lambda x: np.digitize(x['tmid'], bins=tranges, right=True)) \
        .groupby([view_type, 'proc_name', 'io_cat', 'acc_pat', 'func_id']) \
        .agg({
            'duration': ['sum'],
            'size': ['min', 'max', 'sum'],
            'index': ['count'],
            'proc_name': [list],
            'trange': [list],
        })

    sample_1m_bot_df.columns = ['_'.join(col) for col in sample_1m_bot_df.columns.values]
    sample_1m_bot_df.reset_index(inplace=True)

    _set_derived_columns(ddf=sample_1m_bot_df)


    sample_1m_bot_df.drop(columns=['io_cat', 'acc_pat', 'func_id'], inplace=True)

    proc_agg_dict = _get_agg_dict(view_columns=sample_1m_bot_df.columns, is_proc=True)
    non_proc_agg_dict = _get_agg_dict(view_columns=sample_1m_bot_df.columns, is_proc=False)

    sample_1m_bot_df = sample_1m_bot_df \
        .groupby(['proc_name', view_type]) \
        .agg(proc_agg_dict) \
        .groupby([view_type]) \
        .agg(non_proc_agg_dict)

    sample_1m_bot_df['duration_ratio'] = sample_1m_bot_df['duration_sum'] / max_io_time
    sample_1m_bot_df['md_time_ratio'] = sample_1m_bot_df['metadata_time'] / sample_1m_bot_df['duration_sum']

    return sample_1m_bot_df.query(f"duration_ratio > {METADATA_ACCESS_DURATION_THRESHOLD} and md_time_ratio > {METADATA_ACCESS_RATIO_THRESHOLD}"), max_io_time

time: 5.04 ms (started: 2023-06-12 14:07:30 -07:00)


### File-based - Small Writes - 1m Records

In [21]:
!du -ah ../tmp/eval-samples/data-reduction-1m-pandas

30M	../tmp/eval-samples/data-reduction-1m-pandas/full.parquet
30M	../tmp/eval-samples/data-reduction-1m-pandas
time: 280 ms (started: 2023-06-12 13:09:14 -07:00)


In [30]:
%%timeit -r 10

file_name_bott_1m_df, max_io_time = analyze_pandas_small_writes(log_file='../tmp/eval-samples/data-reduction-1m-pandas/full.parquet', view_type='file_name')

2.5 s ± 66.2 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 27.6 s (started: 2023-06-12 13:11:58 -07:00)


In [99]:
file_name_bott_1m_df, max_io_time = analyze_pandas_small_writes(log_file='../tmp/eval-samples/data-reduction-1m-pandas/full.parquet', view_type='file_name')

file_name_bott_1m_df

,duration_sum,size_min,size_max,size_sum,index_count,proc_name_list,trange_list,read_time,write_time,metadata_time,...,open_time,seek_time,stat_time,close_count,open_count,seek_count,stat_count,write_time_ratio,write_avg_size,duration_ratio
file_name,,,,,,,,,,,,,,,,,,,,,
/p/gpfs1/iopp/temp/montage/2854964/5/run/ngc3372.fits,0.56511,0,172352,324021760,1888,"[mAddMPI#lassen576#16133#35184372406720, mAddM...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.0,0.564427,0.000683,...,0.00035,0.0,0.0,4,4,0,0,1.0,172352.0,0.792314


time: 4.69 s (started: 2023-06-12 14:10:42 -07:00)


### File-based - Small Writes - 5m Records

In [22]:
!du -ah ../tmp/eval-samples/data-reduction-5m-pandas

146M	../tmp/eval-samples/data-reduction-5m-pandas/full.parquet
146M	../tmp/eval-samples/data-reduction-5m-pandas
time: 277 ms (started: 2023-06-12 13:09:20 -07:00)


In [100]:
%%timeit -r 10

file_name_bott_5m_df, max_io_time = analyze_pandas_small_writes(log_file='../tmp/eval-samples/data-reduction-5m-pandas/full.parquet', view_type='file_name')

26.2 s ± 1.18 s per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 4min 55s (started: 2023-06-12 14:10:48 -07:00)


In [101]:
file_name_bott_5m_df, max_io_time = analyze_pandas_small_writes(log_file='../tmp/eval-samples/data-reduction-5m-pandas/full.parquet', view_type='file_name')

file_name_bott_5m_df

,duration_sum,size_min,size_max,size_sum,index_count,proc_name_list,trange_list,read_time,write_time,metadata_time,...,open_time,seek_time,stat_time,close_count,open_count,seek_count,stat_count,write_time_ratio,write_avg_size,duration_ratio
file_name,,,,,,,,,,,,,,,,,,,,,
/p/gpfs1/iopp/temp/montage/2854964/5/run/ngc3372.fits,2.825551,0,172352,1620108800,9440,"[mAddMPI#lassen576#16133#35184372406720, mAddM...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.0,2.822133,0.003418,...,0.001751,0.0,0.0,20,20,0,0,1.0,172352.0,0.792314


time: 26.7 s (started: 2023-06-12 14:15:44 -07:00)


### Time-based - Metadata Access - 1m Records

In [96]:
%%timeit -r 10

trange_bott_1m_df, max_io_time = analyze_pandas_metadata_access(log_file='../tmp/eval-samples/data-reduction-1m-pandas/full.parquet', view_type='trange')

3.85 s ± 550 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 43.8 s (started: 2023-06-12 14:08:41 -07:00)


In [97]:
trange_bott_1m_df, max_io_time = analyze_pandas_metadata_access(log_file='../tmp/eval-samples/data-reduction-1m-pandas/full.parquet', view_type='trange')

trange_bott_1m_df

,duration_sum,size_min,size_max,size_sum,index_count,proc_name_list,trange_list,read_time,write_time,metadata_time,...,close_time,open_time,seek_time,stat_time,close_count,open_count,seek_count,stat_count,duration_ratio,md_time_ratio
trange,,,,,,,,,,,,,,,,,,,,,
4,0.201307,-4,32768,209251785,46278,"[mAddMPI#lassen246#41422#35184372406720, mAddM...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",0.073773,0.130431,0.057102,...,0.007963,0.056690,0.000598,0.000415,223,250,162,44,0.282243,0.283657
7,0.088062,-4,32768,141400124,32749,"[mProjExec#lassen246#41328#35184372402064, mPr...","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...",0.014032,0.084562,0.020936,...,0.003913,0.020758,0.000373,0.000099,161,174,102,10,0.123467,0.237740
9,0.145615,-4,32768,190549450,46901,"[mProjExec#lassen246#41328#35184372402064, mPr...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...",0.016931,0.123481,0.044911,...,0.034262,0.016806,0.000463,0.000135,196,224,126,14,0.204160,0.308425
10,0.143008,-4,32768,207868939,51305,"[mProjExec#lassen246#41328#35184372402064, mPr...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...",0.018035,0.135189,0.032654,...,0.006730,0.032256,0.000491,0.000146,208,239,135,15,0.200505,0.228339
11,0.109073,-4,32768,168742324,41405,"[mProjExec#lassen365#41843#35184372402064, mPr...","[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1...",0.014811,0.109007,0.020422,...,0.004668,0.020407,0.000415,0.000129,177,203,114,13,0.152927,0.187235
13,0.123705,-4,32768,163433774,43223,"[mProjExec#lassen246#41328#35184372402064, mPr...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...",0.011930,0.115749,0.031493,...,0.004262,0.031493,0.000344,0.000118,146,181,93,12,0.173442,0.254579
15,0.115701,-4,32768,176738602,42505,"[mProjExec#lassen246#41328#35184372402064, mPr...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...",0.016492,0.111775,0.021066,...,0.005850,0.020854,0.000449,0.000127,193,214,123,13,0.162219,0.182073
18,0.131167,-4,32768,170311970,41322,"[mProjExec#lassen365#41843#35184372402064, mPr...","[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",0.015736,0.107912,0.047644,...,0.005127,0.047430,0.000432,0.000126,182,208,117,13,0.183904,0.363230


time: 4.23 s (started: 2023-06-12 14:09:25 -07:00)


### Time-based - Metadata Access - 5m Records

In [102]:
%%timeit -r 10

trange_bott_5m_df, max_io_time = analyze_pandas_metadata_access(log_file='../tmp/eval-samples/data-reduction-5m-pandas/full.parquet', view_type='trange')

27.5 s ± 2.22 s per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 5min 3s (started: 2023-06-12 14:16:11 -07:00)


In [103]:
trange_bott_5m_df, max_io_time = analyze_pandas_metadata_access(log_file='../tmp/eval-samples/data-reduction-5m-pandas/full.parquet', view_type='trange')

trange_bott_5m_df

,duration_sum,size_min,size_max,size_sum,index_count,proc_name_list,trange_list,read_time,write_time,metadata_time,...,close_time,open_time,seek_time,stat_time,close_count,open_count,seek_count,stat_count,duration_ratio,md_time_ratio
trange,,,,,,,,,,,,,,,,,,,,,
4,1.006534,-4,32768,1046258925,231390,"[mAddMPI#lassen246#41422#35184372406720, mAddM...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",0.368865,0.652157,0.285511,...,0.039816,0.283451,0.002989,0.002075,1115,1250,810,220,0.282243,0.283657
7,0.440308,-4,32768,707000620,163745,"[mProjExec#lassen246#41328#35184372402064, mPr...","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...",0.070159,0.422810,0.104679,...,0.019567,0.103788,0.001866,0.000496,805,870,510,50,0.123467,0.237740
9,0.728076,-4,32768,952747250,234505,"[mProjExec#lassen246#41328#35184372402064, mPr...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...",0.084655,0.617402,0.224557,...,0.171313,0.084028,0.002315,0.000675,980,1120,630,70,0.204160,0.308425
10,0.715041,-4,32768,1039344695,256525,"[mProjExec#lassen246#41328#35184372402064, mPr...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...",0.090176,0.675944,0.163272,...,0.033650,0.161281,0.002453,0.000731,1040,1195,675,75,0.200505,0.228339
11,0.545367,-4,32768,843711620,207025,"[mProjExec#lassen365#41843#35184372402064, mPr...","[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1...",0.074055,0.545034,0.102112,...,0.023338,0.102037,0.002077,0.000647,885,1015,570,65,0.152927,0.187235
13,0.618527,-4,32768,817168870,216115,"[mProjExec#lassen246#41328#35184372402064, mPr...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...",0.059649,0.578745,0.157464,...,0.021311,0.157464,0.001718,0.000590,730,905,465,60,0.173442,0.254579
15,0.578506,-4,32768,883693010,212525,"[mProjExec#lassen246#41328#35184372402064, mPr...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...",0.082460,0.558873,0.105330,...,0.029250,0.104269,0.002247,0.000635,965,1070,615,65,0.162219,0.182073
18,0.655837,-4,32768,851559850,206610,"[mProjExec#lassen365#41843#35184372402064, mPr...","[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",0.078681,0.539562,0.238220,...,0.025637,0.237152,0.002162,0.000629,910,1040,585,65,0.183904,0.363230


time: 28.5 s (started: 2023-06-12 14:21:16 -07:00)
